**8.	Is there a difference in the number of double faults based on gender?**

Importing required libraries

In [4]:
import pandas as pd
import numpy as np

Reading requierd tables and dropping duplicated data

In [7]:
MatchEventInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchEventInfo.csv')
MatchHomeTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchHomeTeamInfo.csv')
MatchAwayTeamInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/MatchAwayTeamInfo.csv')
PeriodInfo = pd.read_csv( 'C:/Users/Maedeh/Desktop/TenisMatchPro/data/external/PeriodInfo.csv')
MatchHomeTeamInfo_dd = MatchHomeTeamInfo.drop_duplicates (subset = 'match_id')
MatchAwayTeamInfo_dd = MatchAwayTeamInfo.drop_duplicates (subset = 'match_id')
MatchEventInfo_dd = MatchEventInfo.drop_duplicates ()


In [ ]:
matches = pd.DataFrame( PeriodInfo['match_id'].unique() , columns = ['match_id'])
ace_data = PeriodInfo[(PeriodInfo['statistic_name'] == 'aces') &
                      (PeriodInfo['period'] == 'ALL')]
ace_data['ace'] = ace_data['home_stat'] + ace_data['away_stat']
ace_data_dd = ace_data.drop_duplicates(subset = 'match_id')
ace_data_dd['ace'] = pd.to_numeric (ace_data_dd['ace'])
ace_data_dd.head()

In [49]:
# Filter rows where the statistic_name is "double_faults" and ALL period
double_faults_data = PeriodInfo[(PeriodInfo['statistic_name'] == 'double_faults') &
                                (PeriodInfo['period'] == 'ALL')]

# Merge with MatchHomeTeamInfo to get home team gender and double faults
home_merged = pd.merge(double_faults_data,
                       MatchHomeTeamInfo_dd[['match_id', 'gender']], on='match_id').drop_duplicates(subset='match_id')
home_double_faults = home_merged[['gender',
                                  'home_stat']].rename(columns={'home_stat': 'double_faults'})

# Merge with MatchAwayTeamInfo to get away team gender and double faults
away_merged = pd.merge(double_faults_data,
MatchAwayTeamInfo_dd[['match_id', 'gender']], on='match_id').drop_duplicates(subset='match_id')
away_double_faults = away_merged[['gender', 
                                  'away_stat']].rename(columns={'away_stat': 'double_faults'})

# Combine home and away double faults
combined_double_faults = pd.concat([home_double_faults, away_double_faults])

# Convert double_faults to numeric
combined_double_faults['double_faults'] = pd.to_numeric(combined_double_faults['double_faults'])
combined_double_faults.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11416 entries, 0 to 11110
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gender         11395 non-null  object
 1   double_faults  11416 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 267.6+ KB


Filter rows where the statistic_name is "double_faults" and Merge with MatchHomeTeamInfo and MatchAwayTeamInfo to get gender and double faults

In [25]:
t=PeriodInfo[PeriodInfo['statistic_name'] == 'double_faults'].groupby(['match_id'], as_index=False)['statistic_name'].count()
t['match_id']

0       12260075
1       12260076
2       12260077
3       12260078
4       12260080
          ...   
6792    12384975
6793    12385017
6794    12385869
6795    12385873
6796    12386383
Name: match_id, Length: 6797, dtype: int64

In [32]:
double_faults_data = PeriodInfo[PeriodInfo['statistic_name'] == 'double_faults'].groupby(['match_id'],
                                                                                         as_index=False)['statistic_name'].count()

merged_HW = MatchHomeTeamInfo_dd[['match_id' ,
                                  'player_id',
                                  'gender']].merge(MatchAwayTeamInfo_dd[['match_id' ,
                                                                          'player_id', 
                                                                          'gender']],
                                                                           on ='match_id').drop_duplicates (subset = 'match_id')
merged_HW_dd = merged_HW.drop_duplicates(subset = 'player_id')
merged_dHW = pd.merge(double_faults_data, merged_HW_dd[['match_id', 'gender' ]], on='match_id')

KeyError: Index(['player_id'], dtype='object')

**Calculating number of players'win**

droping duplicated data

Merging the tables

In [ ]:
merged_HW = MatchHomeTeamInfo_dd[['match_id' ,
                                  'player_id',
                                  'country']].merge(MatchAwayTeamInfo_dd[['match_id' ,
                                                                          'player_id', 
                                                                          'country']],
                                                                           on ='match_id').drop_duplicates (subset = 'match_id')
merged_EHW = merged_HW[['match_id' ,
                        'player_id_x',
                        'country_x',
                        'player_id_y',
                        'country_y']].merge(MatchEventInfo_dd[['match_id',
                                                               'winner_code']],
                                                                on ='match_id').drop_duplicates (subset = 'match_id')
merged_EHW = merged_EHW.dropna()
merged_EHW.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4349 entries, 0 to 5252
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   match_id     4349 non-null   int64  
 1   player_id_x  4349 non-null   int64  
 2   country_x    4349 non-null   object 
 3   player_id_y  4349 non-null   int64  
 4   country_y    4349 non-null   object 
 5   winner_code  4349 non-null   float64
dtypes: float64(1), int64(3), object(2)
memory usage: 237.8+ KB


Mapping the winner_country values

In [ ]:
merged_EHW['winner_country'] = merged_EHW.apply( lambda x : x['country_x']
                                                 if x['winner_code']==1 
                                                else x['country_y'] , axis = 1)
merged_EHW.head()

,match_id,player_id_x,country_x,player_id_y,country_y,winner_code,winner_id,winner_country
0,12260075,14486,Spain,254742,Czech Republic,2.0,254742,Czech Republic
1,12260076,206570,Italy,90080,Russia,1.0,206570,Italy
2,12260077,46391,Germany,275923,Spain,2.0,275923,Spain
3,12260078,106755,Russia,275923,Spain,1.0,106755,Russia
4,12260080,221012,Argentina,57163,Germany,1.0,221012,Argentina


Find the country with the highest total prize money earned

In [ ]:
country = merged_EHW.groupby(['winner_country'])['match_id'].count().idxmax()
max = merged_EHW.groupby(['winner_country'])['match_id'].count().max()
print(f"The country that has produced the most successful tennis players based on total prize money earned is {country} with total prize money of ${max}.")

The country that has produced the most successful tennis players based on total prize money earned is USA with total prize money of $374.
